![](graddesc.jpg)

In [1]:
!which python

/home/tonydevs/miniforge3/envs/fast/bin/python


In [2]:
conda env list

# conda environments:
#
base                     /home/tonydevs/miniforge3
fast                  *  /home/tonydevs/miniforge3/envs/fast
fastai                   /home/tonydevs/miniforge3/envs/fastai


Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch

In [4]:
torch.tensor([1,1,1])

tensor([1, 1, 1])

In [5]:
import numpy as np
from functools import partial
from ipywidgets import interact
import torch
import matplotlib.pyplot as plt

In [6]:
def quad(a,b,c,x): return a*x**2 + b*x + c
def quad_abc_fn(a,b,c): return partial(quad,a,b,c)

In [7]:
# a=3
# b=2
# c=1

@interact(a=3,b=2,c=1)
def plot_quad(a,b,c):
    xs_plotting = torch.linspace(-2.1, 2.1, steps=100)
    quad_model_fn = quad_abc_fn(a,b,c)
    ys_plotting = quad_model_fn(xs_plotting)
    plt.ylim((-1,15))
    plt.xlim((-2.1,2.1))
    plt.plot(xs_plotting, ys_plotting)

    # Save the plot as an image
    plt.savefig("quad_plot1.png")

interactive(children=(IntSlider(value=3, description='a', max=9, min=-3), IntSlider(value=2, description='b', …

![](quad_plot1.png)

In [8]:
# 1. get 20 samples of actual data
quad_321_model = quad_abc_fn(3,2,1)
xs_actual = xs_321_20_inputs_tsr = torch.linspace(-2,2,steps=20)
ys_actual = ys_321_20_actual_tsr = quad_321_model(xs_321_20_inputs_tsr)

np.random.seed(42)

def noise(tsr, scale): return np.random.normal(scale=scale, size=tsr.shape)
def add_noise(tsr,scale,add):
    scale_noise = tsr*noise(tsr, scale)
    added_noise = noise(tsr, scale)
    total_noise = scale_noise + added_noise
    tsr_noisify = tsr + total_noise 
    return tsr_noisify

# 2. add noise to 20 samples of actual data
ys_noisey = ys_321_20_noisey_tsr = add_noise(ys_actual, 0.15, 0.5)

In [9]:
# 3. cal mae
def mae_calc(actual, preds): return torch.abs(actual-preds).mean()


In [10]:
@interact(a=1.1,b=1.1,c=1.1)
def plot_all(a,b,c):
    plot_quad(a,b,c)
    plt.scatter(xs_actual,ys_noisey)
    ys_preds = quad_abc_fn(a,b,c)(xs_actual)
    mae = mae_calc(ys_noisey, ys_preds)

    plt.title(label=f"MAE: {mae:.2f}")
    # Save the plot as an image
    plt.savefig("quad_plot2.png")

interactive(children=(FloatSlider(value=1.1, description='a', max=3.3000000000000003, min=-1.1), FloatSlider(v…

![](quad_plot2.png)

In [11]:
def loss_function(params):
    model = quad_abc_fn(*params)
    ys_preds = model(xs_actual)
    return mae_calc(ys_noisey, ys_preds)

loss_function([1.1,1.1,1.1])

tensor(2.3439, dtype=torch.float64)

In [12]:
abc_tsr = torch.tensor([1.1,1.1,1.1])
abc_tsr

tensor([1.1000, 1.1000, 1.1000])

In [13]:
abc_tsr.requires_grad_()

tensor([1.1000, 1.1000, 1.1000], requires_grad=True)

In [14]:
loss = loss_function(abc_tsr)
loss

tensor(2.3439, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [15]:
loss.backward()

In [16]:
abc_tsr.grad

tensor([-1.4615, -0.0316, -0.7000])

In [17]:
with torch.no_grad():
    abc_tsr -= abc_tsr.grad * 0.01
    loss = loss_function(abc_tsr)
    print(loss)

tensor(2.3176, dtype=torch.float64)


In [18]:
for i in range(10):
    loss = loss_function(abc_tsr)
    loss.backward()
    with torch.no_grad():
        abc_tsr -= 0.01 * abc_tsr.grad
    print(loss, abc_tsr, abc_tsr.grad)

tensor(2.3176, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor([1.1438, 1.1009, 1.1210], requires_grad=True) tensor([-2.9230, -0.0632, -1.4000])
tensor(2.2651, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor([1.1877, 1.1019, 1.1420], requires_grad=True) tensor([-4.3845, -0.0947, -2.1000])
tensor(2.1862, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor([1.2461, 1.1032, 1.1700], requires_grad=True) tensor([-5.8460, -0.1263, -2.8000])
tensor(2.0822, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor([1.3189, 1.1053, 1.2040], requires_grad=True) tensor([-7.2798, -0.2105, -3.4000])
tensor(1.9572, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor([1.4061, 1.1082, 1.2440], requires_grad=True) tensor([-8.7136, -0.2947, -4.0000])
tensor(1.8138, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor([1.5070, 1.1113, 1.2890], requires_grad=True) tensor([-10.0931,  -0.3053,  -4.5000])
tensor(1.6553, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor([1.6212, 1.1152, 1.3380], req

In [19]:
abc_tsr

tensor([2.0392, 1.1359, 1.5050], requires_grad=True)

In [20]:
a,b,c = abc_tsr.tolist()

@interact(a=a,b=b,c=c)
def plot_all_after_gradient_descent(a,b,c):
    plot_quad(a,b,c)
    plt.scatter(xs_actual,ys_noisey)
    ys_preds = quad_abc_fn(a,b,c)(xs_actual)
    mae = mae_calc(ys_noisey, ys_preds)

    plt.title(label=f"MAE: {mae:.2f}")
    # Save the plot as an image
    plt.savefig("quad_plot3.png")

interactive(children=(FloatSlider(value=2.039224147796631, description='a', max=6.117672443389893, min=-2.0392…

![](quad_plot3.png)  

See mae reduced to 0.94 after gradient descent